In [2]:
import numpy as np
import librosa
import os
from glob import glob 
import shutil
import matplotlib.pyplot as plt
import soundfile as sf
import pandas as pd
import json

In [3]:
os.chdir('C:\\Users\Alec\Desktop\GE_Mod_Intern_Workshop\WorkBench_2024\DataStores\Acoustic\AnechoicChamber_stuff\\audio')

In [3]:
files = glob('*.wav')

In [4]:
destination = "C:\\Users\Alec\Desktop\SIT_acoustic\\anechoic"

In [6]:
print(len(files))
count=0
for file in files:
    shutil.copy(file, destination)
    count+=1
print(f"Copied {count} files to {destination}")

544
Copied 544 files to C:\Users\Alec\Desktop\SIT_acoustic\anechoic


In [7]:
%%time
os.chdir("C:\\Users\Alec\Desktop\SIT_acoustic\\anechoic")

filesToProcess = glob('*.wav')

for w in filesToProcess:
    # Load .wav file using librosa
    y, sr = librosa.load(w, sr=44100)
    
    # Calculate number of 1-second chunks
    duration = librosa.get_duration(y=y, sr=sr)
    num_chunks = int(np.ceil(duration))
    
    # Iterate through each chunk
    for i in range(num_chunks):
        # Set the start and end time for the chunk
        start_time = i
        end_time = min(i + 1, duration)
        
        # Extract audio segment for the chunk
        y_chunk = y[int(start_time * sr):int(end_time * sr)]
        
        # Create output file name
        output_file = f"{os.path.splitext(w)[0]}_chunk{i+1}.wav"
        
        # Write chunk data to a new .wav file using soundfile
        sf.write(output_file, y_chunk, sr)
        
    
    # Delete original .wav file
    os.remove(w)
    print(f"Finished processing {w}, the original file has been deleted")

Finished processing DJIM600-FT-A-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-A-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-B-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-B-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-C-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-C-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-D-2426_20191126.wav, the original file has been deleted
Finished processing DJIM600-FT-D-2520_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-A-2216_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-A-2407_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-A-2499_20191126.wav, the original file has been deleted
Finished processing DJIM600-HT-B-2216_20191

In [8]:
from librosa.util import fix_length
chunks = glob('*.wav')
sr = 44100 
targetDuration = 1
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration <= 0.05:
        os.remove(c)
        print(f'Removed file {c} which had duration {duration}')
    elif duration < 1:
        padded = fix_length(y, size=targetDuration*sr)
        sf.write(c, padded, sr)
        print(f'Padded {c}, new duration = {librosa.get_duration(y=padded, sr=44100)}')

Padded DJIM600-FT-A-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-A-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-FT-B-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-B-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-FT-C-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-C-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-FT-D-2426_20191126_chunk23.wav, new duration = 1.0
Padded DJIM600-FT-D-2520_20191126_chunk21.wav, new duration = 1.0
Padded DJIM600-HT-A-2216_20191126_chunk17.wav, new duration = 1.0
Padded DJIM600-HT-A-2407_20191126_chunk18.wav, new duration = 1.0
Padded DJIM600-HT-A-2499_20191126_chunk20.wav, new duration = 1.0
Padded DJIM600-HT-B-2216_20191126_chunk17.wav, new duration = 1.0
Padded DJIM600-HT-B-2407_20191126_chunk18.wav, new duration = 1.0
Padded DJIM600-HT-B-2499_20191126_chunk20.wav, new duration = 1.0
Padded DJIM600-HT-C-2216_20191126_chunk17.wav, new duration = 1.0
Padded DJI

In [9]:
chunks = glob('*.wav')
for c in chunks:
    y, sr = librosa.load(c, sr=44100)
    duration = librosa.get_duration(y=y, sr=sr)
    if duration != 1:
        print(f'{c} has duration {duration} and not duration 1')

In [19]:
# Function to determine prefix number based on file name
def get_prefix_number(filename):
    # Define your prefixes and their corresponding numbers
    prefixes = {
        'DJIM600': 0,
        'DJIS1000': 1,
        'DJIS1000LOUD': 2,
        'MAVIC2PRO': 3,
        'MAVICPRO': 4,
        'MAVICPROPLAT': 5,
        'PHANTOM4': 6,
        'PHANTOM4QP': 7,
    }
    # Iterate through prefixes to find a match
    for prefix, number in prefixes.items():
        if filename.split('-', 1)[0] == prefix:
            return number
    # Return 6 if no prefix matches (assuming there are only 6 prefixes)
    return 8

In [88]:
# List all files in the directory
files = os.listdir(os.getcwd())

# Create lists to store filenames and corresponding prefix numbers
filenames = []
labels = []

# Iterate through files to collect filename and prefix number
for filename in files:
    filenames.append(filename)
    labels.append(get_prefix_number(filename))

# Create a DataFrame
df = pd.DataFrame({
    'Filename': filenames,
    'Label': labels
})

# Specify the output Excel file path
output_excel_file = 'C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx'

# Write DataFrame to Excel
df.to_excel(output_excel_file, index=False)

print(f"Data has been written to {output_excel_file}")

Data has been written to C:\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx


In [22]:
def extract_features(chunk, sr):   
    S = librosa.feature.melspectrogram(y=chunk, sr=sr, n_mels=128)
    chroma = librosa.feature.chroma_stft(S=np.abs(librosa.stft(S, n_fft=256)), sr=sr)
    mfcc = librosa.feature.mfcc(y=chunk, sr=sr, n_mfcc=13)
    mfcc_delta = librosa.feature.delta(mfcc)
    mfcc_delta2 = librosa.feature.delta(mfcc, order=2)

    flat_S = S.flatten()
    flat_chroma = chroma.flatten()
    flat_mfcc = mfcc.flatten()
    flat_mfcc_delta = mfcc_delta.flatten()
    flat_mfcc_delta2 = mfcc_delta2.flatten()
    
    features = np.hstack((flat_S, flat_chroma, flat_mfcc, flat_mfcc_delta, flat_mfcc_delta2))
    
    return features

In [23]:
def norm_get_feats(files):
    feats = []
    for f in files:
        y, sr = librosa.load(f, sr=44100)
        y = librosa.util.normalize(y)
        feats.append(extract_features(y,sr))
    return feats

In [86]:
features = norm_get_feats(files)

C:\Users\Alec\.conda\envs\slopeWork\Lib\site-packages\librosa\core\spectrum.py:266: UserWarning: n_fft=256 is too large for input signal of length=87
  warnings.warn(
C:\Users\Alec\.conda\envs\slopeWork\Lib\site-packages\librosa\core\pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


In [87]:
df1 = pd.DataFrame(features)

In [89]:
df1

,0,1,2,3,4,5,6,7,8,9,...,17591,17592,17593,17594,17595,17596,17597,17598,17599,17600
0,1.565010,9.228539,15.330981,15.805288,23.153942,23.764130,18.379932,22.899494,17.306631,26.913389,...,1.140124,1.092072,0.403587,-0.243864,-0.811111,-0.990252,-0.990252,-0.990252,-0.990252,-0.990252
1,0.810654,1.708859,2.157921,2.509015,2.933541,2.594192,4.138823,2.396913,3.754083,2.964045,...,0.281643,0.210835,0.309461,0.530429,0.338854,-0.421516,-0.421516,-0.421516,-0.421516,-0.421516
2,0.763484,0.625721,0.289867,1.177902,1.756317,1.307760,0.730610,1.005927,1.351978,0.784658,...,-0.154802,0.007862,-0.147195,-0.379425,-0.244221,-0.220155,-0.220155,-0.220155,-0.220155,-0.220155
3,1.880555,1.794558,1.805314,2.443341,3.242125,3.320523,1.920370,1.457476,2.298863,1.780853,...,-0.406827,-0.599746,-0.077847,0.313326,0.709707,0.766467,0.766467,0.766467,0.766467,0.766467
4,4.295770,3.467466,3.462974,5.793205,1.816781,3.352140,3.389903,1.821062,2.987026,2.126263,...,0.770807,0.284593,-0.648381,-0.911495,-0.697925,0.010130,0.010130,0.010130,0.010130,0.010130
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9251,729.903442,2101.232422,1906.214111,1408.837158,2312.865967,2571.756348,2220.990723,2197.397217,1625.915527,2199.247070,...,0.422014,0.311639,0.395588,0.322014,-0.206937,-0.496778,-0.496778,-0.496778,-0.496778,-0.496778
9252,3462.405518,4710.499023,3600.712158,3441.379395,2517.922119,3142.907715,4123.245117,3198.322021,2580.447998,2038.993164,...,0.184844,0.273126,0.026617,0.132349,0.202704,-0.213678,-0.213678,-0.213678,-0.213678,-0.213678
9253,205.192780,894.039062,2170.957764,2029.999146,1685.711670,1986.284912,2056.052246,2148.876465,3474.798340,3537.837158,...,0.325121,-0.363419,-0.613896,-0.433569,-0.293959,-0.104325,-0.104325,-0.104325,-0.104325,-0.104325
9254,155.000687,977.382874,1462.106812,1302.862305,1461.551270,1196.073975,523.968018,524.938904,744.620850,911.366821,...,0.285659,0.363554,0.213731,-0.297564,-0.340870,-0.293140,-0.293140,-0.293140,-0.293140,-0.293140


In [40]:
# excel_file = 'C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx'

# # Read the Excel file into a DataFrame
# df = pd.read_excel(excel_file)

# # Serialize large arrays to JSON strings
# serialized_arrays = [json.dumps(arr.tolist()) for arr in features]

# # Add a new column 'New Column' to the DataFrame and populate it with serialized_arrays
# df['Features'] = serialized_arrays

# # Specify the output Excel file path
# output_excel_file = 'C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx'

# # Write the updated DataFrame to Excel
# df.to_excel(output_excel_file, index=False)

# print(f"Data has been written to {output_excel_file}")

Data has been written to C:\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.xlsx


In [90]:
res = pd.concat([df, df1], axis=1)

In [93]:
output_excel_file = 'C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.csv'

# Write DataFrame to Excel
res.to_csv(output_excel_file, index=False)

In [4]:
test = pd.read_csv('C:\\Users\Alec\Desktop\SIT_acoustic\labeledAnechoicDataset.csv')

In [5]:
test.head()

,Filename,Label,0,1,2,3,4,5,6,7,...,17591,17592,17593,17594,17595,17596,17597,17598,17599,17600
0,DJIM600-FT-A-2426_20191126_chunk1.wav,0,1.565010,9.228539,15.330981,15.805288,23.153942,23.764130,18.379932,22.899494,...,1.140124,1.092072,0.403587,-0.243864,-0.811111,-0.990252,-0.990252,-0.990252,-0.990252,-0.990252
1,DJIM600-FT-A-2426_20191126_chunk10.wav,0,0.810654,1.708859,2.157921,2.509015,2.933541,2.594192,4.138823,2.396913,...,0.281643,0.210835,0.309461,0.530429,0.338854,-0.421516,-0.421516,-0.421516,-0.421516,-0.421516
2,DJIM600-FT-A-2426_20191126_chunk11.wav,0,0.763484,0.625721,0.289867,1.177902,1.756317,1.307760,0.730610,1.005927,...,-0.154802,0.007862,-0.147195,-0.379425,-0.244221,-0.220155,-0.220155,-0.220155,-0.220155,-0.220155
3,DJIM600-FT-A-2426_20191126_chunk12.wav,0,1.880555,1.794558,1.805314,2.443341,3.242125,3.320523,1.920370,1.457476,...,-0.406827,-0.599746,-0.077847,0.313326,0.709707,0.766467,0.766467,0.766467,0.766467,0.766467
4,DJIM600-FT-A-2426_20191126_chunk13.wav,0,4.295770,3.467466,3.462974,5.793205,1.816781,3.352140,3.389903,1.821062,...,0.770807,0.284593,-0.648381,-0.911495,-0.697925,0.010130,0.010130,0.010130,0.010130,0.010130


In [6]:
labels = test['Label']

In [8]:
labels = np.array(labels)

In [10]:
counts = np.ones(8)

In [11]:
for i in labels:
    counts[i] += 1

In [12]:
counts

array([ 933., 1713.,  405., 1241., 1257., 1233., 1237., 1245.])